In [4]:
import umap
import numpy as np
import pandas as pd
from lapjv import lapjv
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import squareform
from scipy.stats import median_abs_deviation
from scipy.cluster.hierarchy import fcluster, linkage

#Self definition
cluster_channels = 5
f_num = 4000 #feature selection, total number: 12548

#Data input
file = "LNN-BRCA"
data_T = pd.read_csv("{}_log2expression-ER_Status+time+relapse+response.csv".format(file), header = 0, index_col = 0)

#Feature Matrix
dataX = data_T.drop(columns = ["ER_Status","time_to_relapse","relapse_(1=True)","response"])

#Feature variance sorting
MAD_list = median_abs_deviation(dataX, axis = 0)
f_sorted = np.argsort(MAD_list)[::-1]
f_name = dataX.columns[f_sorted]

dataX = dataX.reindex(columns = f_name[:f_num])

#Pairwise distance calculation
dataX = dataX.replace([np.nan, np.inf, -np.inf], 0)
distance_matrix = pairwise_distances(dataX.T, metric = 'cosine')
print("--Pairwise distance calculation: Finish!\n")

#Dimension reduction
reducer = umap.UMAP(n_neighbors = 30, min_dist = 0.1, n_components = 2, metric = 'precomputed', random_state = 1)
embedding = reducer.fit_transform(distance_matrix)
print("--Dimension reduction: Finish!\n")

#Save the results
map_frame = pd.DataFrame(embedding, index = dataX.columns, columns = ["umap_f1","umap_f2"])

#Channels split (Hierarchical Cluster)
print("--applying hierarchical clustering ...\n")
Z = linkage(squareform(distance_matrix, checks=False), 'complete')
labels = fcluster(Z, cluster_channels, criterion='maxclust')
map_frame["Subgroup"] = labels

#Grid Assignment
N = len(dataX.columns)

size1 = int(np.ceil(np.sqrt(N)))
size2 = int(np.ceil(N/size1))
grid_size = (size1, size2)
row_col = np.dstack(np.meshgrid(np.linspace(0, size1-1, size1), np.linspace(0, size2-1, size2), indexing = "ij")).reshape(-1, 2)
xy_loc = pd.DataFrame(row_col[:N].astype(int), columns = ["row","column"])

grid = np.dstack(np.meshgrid(np.linspace(0, 1, size1), np.linspace(0, 1, size2), indexing = "ij")).reshape(-1, 2)
grid_map = grid[:N]
cost_matrix = pairwise_distances(grid_map, embedding, metric = "sqeuclidean").astype(np.float64)
cost_matrix = 100000 * (cost_matrix / cost_matrix.max())
row_asses, col_asses, lap_z = lapjv(cost_matrix)

#Feature assignment results based on pairwise distance
map_frame = map_frame.reindex(dataX.columns[row_asses])
map_frame[["row","column"]] = xy_loc.values
#map_frame.to_csv('{}_2D_assigned_{}X{}_fmap_with_subgroup.csv'.format(file,size1,size2), na_rep='NA')
print("--Grid Assignment: Finish!\n")

--Pairwise distance calculation: Finish!

--Dimension reduction: Finish!

--applying hierarchical clustering ...

--Grid Assignment: Finish!



In [5]:
map_frame

,umap_f1,umap_f2,Subgroup,row,column
LRRC31,-3.924838,-3.721242,2,0,0
PIP,-3.875669,-3.672432,2,0,1
CYP4F8,-3.839873,-3.611968,2,0,2
SCGB2A1,-3.861979,-3.601190,2,0,3
SCGB2A2,-3.952854,-3.560882,2,0,4
...,...,...,...,...,...
PPP1R1A,2.487636,-1.929035,2,63,26
PCK1,2.480858,-1.924835,2,63,27
ITGA7,2.477130,-1.917694,2,63,28
RBP4,2.510489,-1.909008,2,63,29
